In [62]:
import numpy as np 
import pandas as pd 
import gc
import time
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import os

In [63]:
#import first 10,000,000 rows of train and all test data

dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        }

train = pd.read_csv('train.csv', dtype=dtypes, nrows=40000000)
test1 = pd.read_csv('train.csv',  skiprows=range(1, 40000000), nrows=10000000, dtype=dtypes)
test = pd.read_csv('test.csv',dtype=dtypes)


In [64]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000000 entries, 0 to 39999999
Data columns (total 8 columns):
ip                 uint32
app                uint16
device             uint16
os                 uint16
channel            uint16
click_time         object
attributed_time    object
is_attributed      uint8
dtypes: object(2), uint16(4), uint32(1), uint8(1)
memory usage: 1.1+ GB


In [65]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 8 columns):
ip                 uint32
app                uint16
device             uint16
os                 uint16
channel            uint16
click_time         object
attributed_time    object
is_attributed      uint8
dtypes: object(2), uint16(4), uint32(1), uint8(1)
memory usage: 276.6+ MB


In [66]:
train.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,83230,3,1,13,379,2017-11-06 14:32:21,NaN,0
1,17357,3,1,19,379,2017-11-06 14:33:34,NaN,0
2,35810,3,1,13,379,2017-11-06 14:34:12,NaN,0
3,45745,14,1,13,478,2017-11-06 14:34:52,NaN,0
4,161007,3,1,13,379,2017-11-06 14:35:08,NaN,0


In [ ]:
#train[train['is_attributed']==1].groupby('ip').count()#.nlargest(5)#sort_values(['ip'], ascending=False) 

In [68]:
test1.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,147296,13,1,18,477,2017-11-07 09:39:40,NaN,0
1,48281,8,1,19,145,2017-11-07 09:39:40,NaN,0
2,5348,27,1,47,153,2017-11-07 09:39:40,NaN,0
3,118229,26,1,18,121,2017-11-07 09:39:40,NaN,0
4,67836,2,1,6,219,2017-11-07 09:39:40,NaN,0


In [69]:
test.head()

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00


In [70]:
#set click_time and attributed_time as timeseries
train['click_time'] = pd.to_datetime(train['click_time'])
train['attributed_time'] = pd.to_datetime(train['attributed_time'])

test1['click_time'] = pd.to_datetime(test1['click_time'])
test1['attributed_time'] = pd.to_datetime(test1['attributed_time'])

test['click_time'] = pd.to_datetime(test['click_time'])

#train['attributed_hour'] = train['attributed_time'].dt.hour
#train['attributed_hour'][train['attributed_hour'].isnull()] = 25
#train['attributed_hour'] = train['attributed_hour'].astype('category')

#test1['attributed_hour'] = test1['attributed_time'].dt.hour
#test1['attributed_hour'][test1['attributed_hour'].isnull()] = 25
#test1['attributed_hour'] = test1['attributed_hour'].astype('category')

train['click_hour'] = train['click_time'].dt.hour.astype('category')

test1['click_hour'] = test1['click_time'].dt.hour.astype('category')

test['click_hour'] = test['click_time'].dt.hour.astype('category')

In [71]:
# time take until download
#train['time_taken'] = train['attributed_time'] - train['click_time']

#test1['time_taken'] = test1['attributed_time'] - test1['click_time']

test['click_id'] = test['click_id'].astype('int')

labels = ['ip', 'app', 'device', 'os', 'channel']
for k in labels:
    train[k] = train[k].astype('category')
    test[k] = test[k].astype('category')
    test1[k] = test1[k].astype('category')

In [72]:
train.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,click_hour
0,83230,3,1,13,379,2017-11-06 14:32:21,NaT,0,14
1,17357,3,1,19,379,2017-11-06 14:33:34,NaT,0,14
2,35810,3,1,13,379,2017-11-06 14:34:12,NaT,0,14
3,45745,14,1,13,478,2017-11-06 14:34:52,NaT,0,14
4,161007,3,1,13,379,2017-11-06 14:35:08,NaT,0,14


In [73]:
test1.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,click_hour
0,147296,13,1,18,477,2017-11-07 09:39:40,NaT,0,9
1,48281,8,1,19,145,2017-11-07 09:39:40,NaT,0,9
2,5348,27,1,47,153,2017-11-07 09:39:40,NaT,0,9
3,118229,26,1,18,121,2017-11-07 09:39:40,NaT,0,9
4,67836,2,1,6,219,2017-11-07 09:39:40,NaT,0,9


In [74]:
test.head()

,click_id,ip,app,device,os,channel,click_time,click_hour
0,0,5744,9,1,3,107,2017-11-10 04:00:00,4
1,1,119901,9,1,3,466,2017-11-10 04:00:00,4
2,2,72287,21,1,19,128,2017-11-10 04:00:00,4
3,3,78477,15,1,13,111,2017-11-10 04:00:00,4
4,4,123080,12,1,13,328,2017-11-10 04:00:00,4


In [75]:
gc.collect()

4807126

In [76]:
X_train = train.drop(columns=['is_attributed', 'attributed_time', 'click_time'])
                              
                              #'time_taken', 'attributed_hour'])

X_test1 = test1.drop(columns=['is_attributed', 'attributed_time', 'click_time'])

y_train = train['is_attributed']

y_test1 = test1['is_attributed']

X_test = test.drop(columns=['click_id', 'click_time'])

In [77]:
gc.collect()

126

In [78]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000000 entries, 0 to 39999999
Data columns (total 6 columns):
ip            category
app           category
device        category
os            category
channel       category
click_hour    category
dtypes: category(6)
memory usage: 499.3 MB


In [79]:
y_train.shape

(40000000,)

In [80]:
X_test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000000 entries, 0 to 9999999
Data columns (total 6 columns):
ip            category
app           category
device        category
os            category
channel       category
click_hour    category
dtypes: category(6)
memory usage: 127.1 MB


In [81]:
y_test1.shape

(10000000,)

In [82]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18790469 entries, 0 to 18790468
Data columns (total 6 columns):
ip            category
app           category
device        category
os            category
channel       category
click_hour    category
dtypes: category(6)
memory usage: 236.3 MB


In [83]:
X_test.head()

,ip,app,device,os,channel,click_hour
0,5744,9,1,3,107,4
1,119901,9,1,3,466,4
2,72287,21,1,19,128,4
3,78477,15,1,13,111,4
4,123080,12,1,13,328,4


In [84]:
gc.collect()

51

In [85]:
import lightgbm as lgb


# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'num_leaves': 31,
    'learning_rate': 0.08,
    'feature_fraction': 0.9,    
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 1,
    'num_boost_round' : 500,
    'early_stopping_rounds' : 30}
#    'device' : 'gpu',
#    'gpu_platform_id' : 1,
#    'gpu_device_id' : 0


print('Start training...')
# train

gbm = lgb.LGBMClassifier(**params)


Start training...


In [86]:
gbm.fit(X_train, y_train, eval_set=[(X_test1, y_test1)], eval_metric= 'binary_logloss')

C:\Users\edwin\Anaconda3\lib\site-packages\lightgbm\engine.py:99: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\edwin\Anaconda3\lib\site-packages\lightgbm\engine.py:104: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's binary_logloss: 0.617168
Training until validation scores don't improve for 30 rounds.
[2]	valid_0's binary_logloss: 0.552406
[3]	valid_0's binary_logloss: 0.496389
[4]	valid_0's binary_logloss: 0.447778
[5]	valid_0's binary_logloss: 0.405037
[6]	valid_0's binary_logloss: 0.36728
[7]	valid_0's binary_logloss: 0.333785
[8]	valid_0's binary_logloss: 0.303969
[9]	valid_0's binary_logloss: 0.27738
[10]	valid_0's binary_logloss: 0.253439
[11]	valid_0's binary_logloss: 0.231864
[12]	valid_0's binary_logloss: 0.212399
[13]	valid_0's binary_logloss: 0.194887
[14]	valid_0's binary_logloss: 0.178942
[15]	valid_0's binary_logloss: 0.164487
[16]	valid_0's binary_logloss: 0.151351
[17]	valid_0's binary_logloss: 0.139372
[18]	valid_0's binary_logloss: 0.128456
[19]	valid_0's binary_logloss: 0.118515
[20]	valid_0's binary_logloss: 0.10942
[21]	valid_0's binary_logloss: 0.10111
[22]	valid_0's binary_logloss: 0.0935811
[23]	valid_0's binary_logloss: 0.0866174
[24]	valid_0's binary_logloss

LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
        class_weight=None, colsample_bytree=1.0, early_stopping_rounds=30,
        feature_fraction=0.9, learning_rate=0.08, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_boost_round=500, num_leaves=31,
        objective='binary', random_state=None, reg_alpha=0.0,
        reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1, verbose=1)

In [87]:
print('Save model...')
# save model to file
gbm.booster_.save_model('LightGBMmodel_2.txt')


Save model...


In [88]:
#gbm = lgb.Booster(model_file='LightGBMmodel.txt')


In [89]:

print('Start predicting...')
# predict
y_pred = gbm.predict_proba(X_test, gbm.best_iteration_)
# eval
#print('The rmse of prediction is:', mean_squared_error(y_test1, y_pred) ** 0.5)


Start predicting...


In [90]:
gc.collect()

321

In [91]:
y_pred.shape

(18790469, 2)

In [92]:
y_pred[:,1]

array([1.01486897e-03, 7.08838283e-04, 1.36849232e-04, ...,
       3.31146191e-04, 9.82210491e-04, 8.72329055e-05])

In [93]:
output = pd.DataFrame(test['click_id'])
output['is_attributed'] = y_pred[:,1]

In [94]:
output.head()

,click_id,is_attributed
0,0,0.001015
1,1,0.000709
2,2,0.000137
3,3,0.000356
4,4,0.000149


In [95]:
output.to_csv("submission_LightGBM_2.csv",index=False)